In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pickle
import hdf5_getters

## Get track path from the data and serialize (comment if you have the serialized list)

In [35]:
# track_base_path = '/mnt/snap/data/'
# tracks_path = os.listdir(track_base_path)
# tracks_path = [track_base_path  + str(x) for x in tracks_path]
# # for path in paths:
# #     all_paths.append(os.listdir(path))

# all_paths = []
# for path in tracks_path:
#     path_list = os.listdir(path)
#     full_path = [path + '/' + str(x) for x in path_list]
#     all_paths.append(full_path)


# temp = []
# tracks = []
# for path in all_paths:
#     for p2 in path:
#         temp.append([p2 + '/' +x  for x in os.listdir(p2)])

# for tp in temp:
#     for i in tp:
#         tracks.append(i)

# track_path = []
# for track in tracks:
#     track_path.append([track + '/' + x for x in os.listdir(track)])
    

In [34]:
# track_paths_norm = []
# for tp in track_path:
#     for i in tp:
#         track_paths_norm.append(i)

In [26]:
# binary_file = open('tracks.bin',mode='wb')
# my_pickled_mary = pickle.dump(track_paths_norm, binary_file)
# binary_file.close()

### Load from binary file

In [2]:
with open("./tracks_fin.bin", "rb") as file:
    track_paths_norm = pickle.load(file)

In [4]:
store = pd.HDFStore(track_paths_norm[0])
keys = store.keys()
print(keys)
store.close()

Opening /mnt/snap/data/Q/G/H/TRQGHIK128F9329CC4.h5 in read-only mode
['/analysis/songs', '/metadata/songs', '/musicbrainz/songs']


In [5]:
keys = ['/analysis/songs', '/metadata/songs', '/musicbrainz/songs']
def create_complete_row(track):
    df = pd.concat([pd.read_hdf(track, key=keys[0]), 
                    pd.read_hdf(track, key=keys[1]),
                    pd.read_hdf(track, key=keys[2])], axis=1)
    
    return df

In [ ]:
df_final = pd.DataFrame()
count1 = 0
count2 = 0
for track in track_paths_norm:
    count1 += 1
    
    df = create_complete_row(track)
    h5 = hdf5_getters.open_h5_file_read(track)
    pitch_df = pd.DataFrame(hdf5_getters.get_segments_pitches(h5).mean(axis=0)).T
    pitch_df.columns = ['pitch_' + str(i) for i in range(12)]
    timbre_df = pd.DataFrame(hdf5_getters.get_segments_timbre(h5).mean(axis=0)).T
    timbre_df.columns = ['timbre_' + str(i) for i in range(12)]
    df = pd.concat((df, pitch_df, timbre_df), axis=1)
    df_final = df_final.append(df)
    h5.close()
    
    if(count1 == 1000):
        count2 += 1
        df_final.to_csv("songs_" + str(count2) + ".csv")
        count1 = 0
        df_final = pd.DataFrame()


In [15]:
df_final.shape

(595, 77)

In [32]:
df.to_csv('songs.csv')

In [8]:
track_paths_norm[0]

'/mnt/snap/data/Q/G/H/TRQGHIK128F9329CC4.h5'

## Looking at tagtraum data

In [33]:
addnl = pd.read_table('./data/tagtraum/msd_tagtraum_cd1.cls/msd_tagtraum_cd1.txt', sep='\t', skiprows=7, names=['track_id', 'Tag1', 'Tag2'])

In [36]:
df_m = df.merge(addnl, on='track_id')

In [45]:
df_m.to_csv('songs_with_genre.csv')